<a href="https://colab.research.google.com/github/dlbkv/AI-Step/blob/master/module3/lab6_lin-reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Student_Performance.csv", index_col="Unnamed: 0")

[Інформація про дані](https://www.kaggle.com/datasets/nikhil7280/student-performance-multiple-linear-regression)

In [ ]:
df.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7.0,99.0,NaN,9.0,1.0,91.0
1,4.0,82.0,No,4.0,2.0,65.0
2,8.0,51.0,Yes,7.0,2.0,45.0
3,5.0,52.0,Yes,5.0,2.0,36.0
4,7.0,75.0,No,8.0,5.0,66.0


# Завдання 1
Продивтіться дані, очистіть від викидів, видаліть пропущені значення з стовпчика `Perfomance Index`

In [ ]:
df.describe()

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Performance Index
count,7316.000000,7316.000000,7316.000000,7316.00000,7316.000000
mean,5.018316,69.376435,6.517223,4.56766,55.204347
std,2.588411,17.239193,1.693413,2.86663,19.161720
min,1.000000,40.000000,4.000000,0.00000,10.000000
25%,3.000000,54.000000,5.000000,2.00000,40.000000
50%,5.000000,69.000000,7.000000,5.00000,55.000000
75%,7.000000,84.000000,8.000000,7.00000,70.250000
max,9.000000,99.000000,9.000000,9.00000,100.000000


In [ ]:
df = df.dropna()


In [ ]:
def remove_outliers(df, column_name):
  column = df[column_name]
  q1 = column.quantile(q=0.25)
  q3 = column.quantile(q=0.75)
  iqr = q3 - q1
  lower = q1 - iqr*1.5
  upper = q3 + iqr*1.5
  mask = (column >= lower) & (column <=upper)
  return df[mask]

In [ ]:
for column_name in df.columns:
  if df.dtypes[column_name] in (int, float):
    df = remove_outliers(df, column_name)


# Завдання 2
Розбийте дані на тренувальні та тестові

In [ ]:
from sklearn.model_selection import train_test_split
y = df['Performance Index']
X = df.drop(columns='Performance Index')
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [ ]:
X

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Extracurricular Activities_No,Extracurricular Activities_Yes
1,4.0,82.0,4.0,2.0,1,0
2,8.0,51.0,7.0,2.0,0,1
3,5.0,52.0,5.0,2.0,0,1
4,7.0,75.0,8.0,5.0,1,0
5,3.0,78.0,9.0,6.0,1,0
...,...,...,...,...,...,...
9991,5.0,97.0,7.0,4.0,0,1
9994,6.0,46.0,8.0,0.0,0,1
9996,7.0,64.0,8.0,5.0,0,1
9998,9.0,97.0,7.0,0.0,0,1


# Завдання 3
Побудуйте `Ppileline` для заповнення пропущених значень в усіч стовпчиках

# Завдання 4
Побудуйте модель та натренуйте її, виведіть метрики

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
from sklearn import metrics
def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["MSE", name] = metrics.mean_squared_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [ ]:
all_metrics = pd.DataFrame()
all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

In [ ]:
all_metrics

,train,test
MAE,1.62,1.58
MSE,4.18,4.03
RMSE,2.04,2.01
R2,0.99,0.99


# Завдання 5 оптимально
Спробуйте покращити модель